In [3]:
import pandas as pd
import requests

# Objective Function
## Pareto Frontier of Sharpe Ratio and ESG Score

The Sharpe Ratio is defined as:

$$
\text{Sharpe Ratio} = \frac{\sum_{i=1}^n w_i \mu_i - R_f}{\sqrt{w^T \Sigma w}}
$$

Where:
- $μ_i$ = Portfolio return  
- $R_f$ = Risk-free rate  
- $σ_p$ = Portfolio volatility (standard deviation)
- $Σw$	= Covariance matrix

In [31]:
# gather ticker info from Alpha Vantage API
API_KEY = "OD76BXNCWU9GO4WP"
symbols = ["MSFT", "AAPL", "NVDA", "UNH", "JNJ", "JPM", "V", "AMZN", "TSLA", "PG", 
           "KO", "HON", "UNP", "NEE", "CVX", "DUK", "PLD", "LIN", "GOOGL", "VZ"]
test = ["MSFT", "NVDA"]

weights = [0.3, 0.7]

esg_scores = [53,61]

def fetch_daily_prices(ticker):
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": ticker,
        "apikey": API_KEY,
        "outputsize": "full"
    }

    response = requests.get(url, params=params)
    data = response.json()

    # Check for valid time series key
    time_series_key = next((key for key in data if "Time Series" in key), None)
    if not time_series_key:
        print(f"[Error] No time series for {ticker}. Response: {data}")
        return pd.Series(dtype=float)  # Empty series

    df = pd.DataFrame(data[time_series_key]).T
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    df["close"] = df["4. close"].astype(float)

    return df["close"].iloc[-1260:]  # Last 5 years of trading days

def fetch_all_prices(symbols, output_file="daily_close_prices.csv"):
    all_prices = {}

    for i, symbol in enumerate(symbols):
        print(f"[{i+1}/{len(symbols)}] Fetching {symbol}...")
        all_prices[symbol] = fetch_daily_prices(symbol)

    df = pd.DataFrame(all_prices)
    df.to_csv(output_file)
    print(f"\n Saved data to {output_file}")
    return df

In [33]:

df_prices = fetch_all_prices(test)

[1/2] Fetching MSFT...
[Error] No time series for MSFT. Response: {'Information': 'Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints'}
[2/2] Fetching NVDA...
[Error] No time series for NVDA. Response: {'Information': 'Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints'}

 Saved data to daily_close_prices.csv


In [37]:
# TODO: Might have to take out the returns calculation and have sharpe ratio calculations seperate

def calculate_portfolio_metrics(prices_df, weights, risk_free_rate=0.02):
    # Step 1: Calculate daily returns
    daily_returns = prices_df.pct_change().dropna()
    
    # Step 2: Calculate mean annual return
    mean_daily_returns = daily_returns.mean()
    mean_annual_returns = (1 + mean_daily_returns) ** 252 - 1

    # Step 3: Calculate annualized covariance matrix
    cov_matrix_annual = daily_returns.cov() * 252

    # Step 4: Compute expected return, volatility, Sharpe ratio
    weights_array = np.array(weights)
    expected_return = np.dot(weights_array, mean_annual_returns)
    portfolio_variance = np.dot(weights_array.T, np.dot(cov_matrix_annual, weights_array))
    portfolio_volatility = np.sqrt(portfolio_variance)
    sharpe_ratio = (expected_return - risk_free_rate) / portfolio_volatility

    return {
        "Expected Annual Return": expected_return,
        "Annual Volatility": portfolio_volatility,
        "Sharpe Ratio": sharpe_ratio
    }

In [ ]:
def eval_esg_scores(esg, weights):
    return np.dot(weights, esg)

In [ ]:
# ABC Algorithm

#Pseudocode: